In [35]:
import requests
import json
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from io import StringIO
import os
import 

SyntaxError: invalid syntax (3243447599.py, line 9)

In [33]:
today = datetime.today().strftime('%Y-%m-%d')
today

'2025-02-01'

In [13]:
songlist_url_api = "https://elgoose.net/api/v2/songs.json?order_by=name"
songlistdata_json = requests.get(songlist_url_api).json()
song_df = pd.DataFrame(songlistdata_json['data'])
songdata_og = song_df.drop(columns=['slug','created_at','updated_at'])

songdata_og.head()

,id,name,isoriginal,original_artist
0,638,(7hunder),1,Goose
1,637,(dawn),1,Goose
2,593,(Marie's The Name) His Latest Flame,0,Elvis Presley
3,636,(satellite),1,Goose
4,617,(You're The) Devil In Disguise,0,Elvis Presley


In [ ]:
songlist_url = "https://elgoose.net/song/"
response = requests.get(songlist_url)
response.raise_for_status()  # Raise an exception for bad status codes
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
tables = soup.find_all('table')
if tables:
    tables_str = str(tables)  # Convert tables to string
    tables_io = StringIO(tables_str)  # Wrap in StringIO
    tables = pd.read_html(tables_io)
songdata_info = tables[1].copy()


songdata_info.head()

,Song Name,Original Artist,Debut Date,Last Played,Times Played Live,Avg Show Gap
0,(dawn),Goose,2022-01-26,2024-12-13,8,30.38
1,(Marie's The Name) His Latest Flame,Elvis Presley,2019-06-28,2022-08-20,4,99.25
2,(satellite),Goose,2018-06-01,2024-09-13,3,163.67
3,(Your Love Keeps Lifting Me) Hig...,Jackie Wilson,2018-10-27,2020-12-11,3,150.33
4,2021,Vampire Weekend,2021-10-08,2021-12-31,3,132.00


In [15]:
songdata = songdata_og.merge(songdata_info, left_on="name", right_on="Song Name", how="inner")[['id','Song Name', 'isoriginal','Original Artist', 'Debut Date','Last Played',\
                                                                                                'Times Played Live', 'Avg Show Gap']]
songdata = songdata.rename(columns={'Song Name': 'song_name', 'Original Artist': 'original_artist', 'Debut Date': 'debut_date', 'Last Played': 'last_played', 'Times Played Live': 'times_played_live', 'Avg Show Gap': 'avg_show_gap'})
songdata.head()

,id,song_name,isoriginal,original_artist,debut_date,last_played,times_played_live,avg_show_gap
0,637,(dawn),1,Goose,2022-01-26,2024-12-13,8,30.38
1,593,(Marie's The Name) His Latest Flame,0,Elvis Presley,2019-06-28,2022-08-20,4,99.25
2,636,(satellite),1,Goose,2018-06-01,2024-09-13,3,163.67
3,709,2021,0,Vampire Weekend,2021-10-08,2021-12-31,3,132.00
4,432,726,1,Goose,2020-06-20,2024-12-13,35,8.83


In [39]:
showlist_url = "https://elgoose.net/api/v2/shows.json?order_by=showdate"
showlistdata_json = requests.get(showlist_url).json()
showlist_df = pd.DataFrame(showlistdata_json['data'])
past_df = showlist_df[showlist_df['showdate'] < today]
future_df = showlist_df[showlist_df['showdate'] >= today].sort_values('showdate').head(1)
result = pd.concat([past_df, future_df])
goose_showlist_df = result[(result['artist'] == 'Goose')].copy().reset_index(drop=True)
goose_showlist_df.tail()

,show_id,showdate,permalink,artist_id,artist,showtitle,venue_id,venuename,location,city,...,tour_id,tourname,showorder,show_year,show_day,show_dayname,show_month,show_monthname,updated_at,created_at
698,1719248965,2024-12-13,goose-december-13-2024-north-charleston-colise...,1,Goose,,488,North Charleston Coliseum &amp; Performing Art...,"North Charleston, SC, USA",North Charleston,...,1,Not Part of a Tour,1,2024,13,Friday,12,December,2024-12-30 22:45:59,2024-06-24 17:09:25
699,1719248984,2024-12-14,goose-december-14-2024-north-charleston-colise...,1,Goose,,488,North Charleston Coliseum &amp; Performing Art...,"North Charleston, SC, USA",North Charleston,...,38,Fall Tour 2024,1,2024,14,Saturday,12,December,2024-12-21 21:28:27,2024-06-24 17:09:44
700,1721059697,2024-12-30,goose-december-30-2024-moody-center-austin-tx-...,1,Goose,,493,Moody Center,"Austin, TX, USA",Austin,...,1,Not Part of a Tour,1,2024,30,Monday,12,December,2025-01-01 08:53:09,2024-07-15 16:08:17
701,1721059705,2024-12-31,goose-december-31-2024-moody-center-austin-tx-...,1,Goose,,493,Moody Center,"Austin, TX, USA",Austin,...,1,Not Part of a Tour,1,2024,31,Tuesday,12,December,2025-01-04 00:13:37,2024-07-15 16:08:25
702,1731950152,2025-02-06,goose-february-6-2025-miller-high-life-theatre...,1,Goose,,370,Miller High Life Theatre,"Milwaukee, WI, USA",Milwaukee,...,39,Winter Tour 2025,1,2025,6,Thursday,2,February,2024-11-18 17:15:52,2024-11-18 17:15:52


In [34]:
showlist_url = "https://elgoose.net/api/v2/shows.json?order_by=showdate"
showlistdata_json = requests.get(showlist_url).json()
showlist_df = pd.DataFrame(showlistdata_json['data'])
goose_showlist_df = showlist_df[(showlist_df['artist'] == 'Goose')&(showlist_df['showdate'] < today)].copy().reset_index(drop=True)

In [17]:
venuedata = goose_showlist_df[['venue_id', 'venuename', 'location', 'city', 'state', 'country']].drop_duplicates().sort_values(by=['venue_id'], ascending=True).reset_index(drop=True)
venuedata.head()

,venue_id,venuename,location,city,state,country
0,1,Madison Theater,"Covington, KY, USA",Covington,KY,USA
1,2,Peach Music Festival,"Scranton, PA, USA",Scranton,PA,USA
2,3,Bowery Ballroom,"New York, NY, USA",New York,NY,USA
3,4,Goose Community Rec Center,"Redding, CT, USA",Redding,CT,USA
4,5,Yarmouth Drive-In,"West Yarmouth, MA, USA",West Yarmouth,MA,USA


In [18]:
tourdata = goose_showlist_df[['tour_id', 'tourname']].drop_duplicates().sort_values(by=['tour_id'], ascending=True).reset_index(drop=True)
tourdata.head()

,tour_id,tourname
0,1,Not Part of a Tour
1,2,Bingo Tour
2,3,Winter Tour 2020
3,4,Live From Out There
4,5,Fall Tour 2019


In [19]:
showdata = goose_showlist_df[['show_id', 'showdate', 'showtitle', 'venue_id', 'tour_id', 'showorder'\
    ]].copy().sort_values(by=['showdate', 'showorder'], ascending=[True, True]).reset_index(names='show_number').assign(show_number= lambda x: x['show_number'] + 1).drop(columns=['showorder'])
showdata['showtitle'] = showdata['showtitle'].replace('', None)

showdata.head()

,show_number,show_id,showdate,showtitle,venue_id,tour_id
0,1,1605930400,2014-09-27,None,24,1
1,2,1616038942,2014-10-24,None,35,1
2,3,1613690100,2014-10-25,None,26,1
3,4,1613690131,2014-11-21,None,25,1
4,5,1616038775,2014-11-26,None,35,1


In [31]:
showdata.tail(25)

,show_number,show_id,showdate,showtitle,venue_id,tour_id
710,711,1737015351,2025-04-24,None,515,1
711,712,1727890080,2025-05-08,None,502,1
712,713,1727890094,2025-05-09,None,502,1
713,714,1727890109,2025-05-10,None,502,1
714,715,1736785860,2025-05-25,None,513,1
715,716,1737998824,2025-05-30,None,518,40
716,717,1738001411,2025-05-31,None,373,40
717,718,1738001727,2025-06-01,None,519,40
718,719,1738001750,2025-06-03,None,297,40
719,720,1738001942,2025-06-04,None,520,40


In [24]:
setlist_url = "https://elgoose.net/api/v1/setlists.json?order_by=showdate"

setlistdata_json = requests.get(setlist_url).json()
setlist_df = pd.DataFrame(setlistdata_json['data'])

In [25]:
transition_data  = setlist_df[['transition_id', 'transition']].drop_duplicates().sort_values(by=['transition_id'], ascending=True).reset_index(drop=True)
transition_data.head()

,transition_id,transition
0,1,","
1,2,>
2,3,->
3,4,
4,5,


In [26]:
setlistdata = setlist_df[setlist_df['artist'] == "Goose"].copy().reset_index(drop=True)[['uniqueid', 'show_id', 'song_id','setnumber','position','tracktime','transition_id','isreprise','isjam',\
                                                                                         'footnote','isjamchart','jamchart_notes','soundcheck','isverified','isrecommended']]

setlistdata.head()

,uniqueid,show_id,song_id,setnumber,position,tracktime,transition_id,isreprise,isjam,footnote,isjamchart,jamchart_notes,soundcheck,isverified,isrecommended
0,40785,1613690131,871,1,1,None,1,0,0,John Scofield. First known version.,0,,,1.0,NaN
1,40786,1613690131,483,1,2,None,1,0,0,First known version.,0,,,1.0,NaN
2,40787,1613690131,569,1,3,None,1,0,0,First known version.,0,,,1.0,NaN
3,40788,1613690131,873,1,4,None,1,0,0,Grateful Dead. First known version.,0,,,1.0,NaN
4,40789,1613690131,400,1,5,None,1,0,0,First known version.,0,,,1.0,NaN


In [27]:
setlistdata[setlistdata['song_id']==593]

,uniqueid,show_id,song_id,setnumber,position,tracktime,transition_id,isreprise,isjam,footnote,isjamchart,jamchart_notes,soundcheck,isverified,isrecommended
1471,5441,1614300844,593,1,16,None,1,0,0,Elvis Presley. FTP.,0,,,0.0,NaN
2326,753,1599321108,593,1,4,8:36,1,0,0,Elvis Presley.,0,None,,1.0,NaN
2674,8782,1627784422,593,1,5,10:46,1,0,0,Elvis Presley.,0,None,Jam > (Marie's The Name) His Latest Flame > Li...,1.0,0.0
3683,12950,1661051264,593,1,4,9:25,1,0,0,Elvis Presley.,0,None,,1.0,NaN


In [5]:
songs_and_shows = pd.merge(setlist,showlist,how="inner", on="show_id")
songs_and_shows['gap'] = songs_and_shows.sort_values(['songname','shownumber']).groupby('songname')['shownumber'].diff()
songs_and_shows.rename({'songname' : 'song'}, axis=1, inplace=True)
songs_and_shows['reprise'] = songs_and_shows.groupby(['shownumber','song']).cumcount()
songs_and_shows = songs_and_shows.drop(['isreprise'], axis=1)
songs_and_shows['showdate'] = pd.to_datetime(songs_and_shows['showdate'])

In [6]:
songs_and_shows_noreprise = songs_and_shows.loc[(songs_and_shows['reprise']==0)]
max_shownumber = songs_and_shows.shownumber.max()
songs_count = songs_and_shows_noreprise.groupby('song').size().to_frame('timesplayed')
songs_ftp_date = songs_and_shows.groupby('song')['showdate'].min().to_frame()
songs_ftp_shownumber = songs_and_shows.groupby('song')['shownumber'].min().to_frame()
songs_ltp_date = songs_and_shows.groupby('song')['showdate'].max().to_frame()
songs_ltp_shownumber = songs_and_shows.groupby('song')['shownumber'].max().to_frame()
songs_current_gap = songs_ltp_shownumber.subtract(max_shownumber).abs()
songs_avg_gap = songs_and_shows_noreprise.groupby('song')['gap'].mean().to_frame()
songs_med_gap = songs_and_shows_noreprise.groupby('song')['gap'].median().to_frame()

In [7]:
songs_and_shows_noreprise_pc=songs_and_shows_noreprise.loc[(songs_and_shows_noreprise['show_year']>2020)]
songs_count_pc = songs_and_shows_noreprise_pc.groupby('song').size().to_frame('timesplayed_postcovid')
songs_avg_gap_pc = songs_and_shows_noreprise_pc.groupby('song')['gap'].mean().to_frame()
songs_med_gap_pc = songs_and_shows_noreprise_pc.groupby('song')['gap'].median().to_frame()

In [8]:
todayminusyear = datetime.now() - relativedelta(years=1)
todayminusyear = todayminusyear.strftime('%Y/%m/%d')
songs_and_shows_noreprise_lastyear =songs_and_shows_noreprise.loc[(songs_and_shows_noreprise['showdate']>todayminusyear)]
songs_and_shows_noreprise_lastyear.reset_index(inplace=True, drop=True)
songs_count_lastyear = songs_and_shows_noreprise_lastyear.groupby('song').size().to_frame('timesplayed_lastyear')
songs_avg_gap_ly = songs_and_shows_noreprise_lastyear.groupby('song')['gap'].mean().to_frame()
songs_med_gap_ly = songs_and_shows_noreprise_lastyear.groupby('song')['gap'].median().to_frame()

In [9]:
songs_current_gap.rename(columns={'ltp_shownumber':'current_gap'}, inplace=True)
songs_ftp_date.rename(columns={'showdate':'ftp_date'}, inplace=True)
songs_ftp_shownumber.rename(columns={'shownumber':'ftp_shownumber'}, inplace=True)
songs_ltp_date.rename(columns={'showdate':'ltp_date'}, inplace=True)
songs_ltp_shownumber.rename(columns={'shownumber':'ltp_shownumber'}, inplace=True)
songs_current_gap.rename(columns={'shownumber':'current_gap'}, inplace=True)
songs_avg_gap.rename(columns={'gap':'avg_gap'}, inplace=True)
songs_med_gap.rename(columns={'gap':'med_gap'}, inplace=True)
songs_avg_gap_pc.rename(columns={'gap':'avg_gap_pc'}, inplace=True)
songs_med_gap_pc.rename(columns={'gap':'med_gap_pc'}, inplace=True)
songs_avg_gap_ly.rename(columns={'gap':'avg_gap_ly'}, inplace=True)
songs_med_gap_ly.rename(columns={'gap':'med_gap_ly'}, inplace=True)

In [10]:
song_stats = songs_count.merge(songs_count_pc, on='song')
song_stats = song_stats.merge(songs_count_lastyear, on='song')
song_stats = song_stats.merge(songs_ftp_date, on='song')
song_stats = song_stats.merge(songs_ftp_shownumber, on='song')
song_stats = song_stats.merge(songs_ltp_date, on='song')
song_stats = song_stats.merge(songs_current_gap, on='song')
song_stats = song_stats.merge(songs_avg_gap, on='song')
song_stats = song_stats.merge(songs_med_gap, on='song')
song_stats = song_stats.merge(songs_avg_gap_pc, on='song')
song_stats = song_stats.merge(songs_med_gap_pc, on='song')
song_stats = song_stats.merge(songs_avg_gap_ly, on='song')
song_stats = song_stats.merge(songs_med_gap_ly, on='song')

In [11]:
RicksNotebook_pc = song_stats.loc[song_stats['timesplayed_postcovid']>0]
RicksNotebook_pc['Gap:Current - Avg'] = RicksNotebook_pc['current_gap'] - RicksNotebook_pc['avg_gap']
RicksNotebook_pc['Gap:Current - Median'] = RicksNotebook_pc['current_gap'] - RicksNotebook_pc['med_gap']
RicksNotebook_pc['Gap:Current - Avg_PC'] = RicksNotebook_pc['current_gap'] - RicksNotebook_pc['avg_gap_pc']
RicksNotebook_pc['Gap:Current - Median_PC'] = RicksNotebook_pc['current_gap'] - RicksNotebook_pc['med_gap_pc']
RicksNotebook_pc = RicksNotebook_pc.sort_values(by=['timesplayed_postcovid', 'timesplayed'], ascending=False).reset_index()
RicksNotebook_pc = RicksNotebook_pc.drop(columns=['timesplayed','ftp_date','ftp_shownumber','avg_gap','med_gap', \
                                              'Gap:Current - Avg','Gap:Current - Median', 'timesplayed_lastyear',\
                                                 'avg_gap_ly','med_gap_ly'])
RicksNotebook_pc3 = RicksNotebook_pc.loc[RicksNotebook_pc['current_gap']>2].reset_index(drop=True)
RicksNotebook_pc3.head(25)

,song,timesplayed_postcovid,ltp_date,current_gap,avg_gap_pc,med_gap_pc,Gap:Current - Avg_PC,Gap:Current - Median_PC
0,Hot Tea,68,2024-11-09,4,4.088235,4.0,-0.088235,0.0
1,All I Need,62,2024-11-08,5,4.467742,4.0,0.532258,1.0
2,Tumble,62,2024-11-10,3,4.516129,4.0,-1.516129,-1.0
3,Arrow,62,2024-11-09,4,4.500000,4.0,-0.500000,0.0
4,Creatures,61,2024-11-07,6,4.557377,4.0,1.442623,2.0
5,Madhuvan,59,2024-11-07,6,4.677966,4.0,1.322034,2.0
6,Flodown,59,2024-11-08,5,4.762712,5.0,0.237288,0.0
7,Rockdale,58,2024-11-10,3,4.368421,4.0,-1.368421,-1.0
8,Slow Ready,55,2024-11-10,3,5.072727,5.0,-2.072727,-2.0
9,Drive,54,2024-11-09,4,5.203704,5.0,-1.203704,-1.0


In [12]:
RicksNotebook_ly = song_stats.loc[song_stats['timesplayed_lastyear']>0]
RicksNotebook_ly['Gap:Current - Avg'] = RicksNotebook_ly['current_gap'] - RicksNotebook_ly['avg_gap']
RicksNotebook_ly['Gap:Current - Median'] = RicksNotebook_ly['current_gap'] - RicksNotebook_ly['med_gap']
RicksNotebook_ly['Gap:Current - Avg_LY'] = RicksNotebook_ly['current_gap'] - RicksNotebook_ly['avg_gap_ly']
RicksNotebook_ly['Gap:Current - Median_LY'] = RicksNotebook_ly['current_gap'] - RicksNotebook_ly['med_gap_ly']
RicksNotebook_ly = RicksNotebook_ly.sort_values(by=['timesplayed_lastyear', 'timesplayed'], ascending=False).reset_index()
RicksNotebook_ly = RicksNotebook_ly.drop(columns=['timesplayed','ftp_date','ftp_shownumber','avg_gap','med_gap', \
                                              'Gap:Current - Avg','Gap:Current - Median', 'timesplayed_postcovid',\
                                                 'avg_gap_pc','med_gap_pc'])
RicksNotebook_ly3 = RicksNotebook_ly.loc[RicksNotebook_ly['current_gap']>2].reset_index(drop=True)
RicksNotebook_ly3.head(25)

,song,timesplayed_lastyear,ltp_date,current_gap,avg_gap_ly,med_gap_ly,Gap:Current - Avg_LY,Gap:Current - Median_LY
0,Hot Love & The Lazy Poet,16,2024-11-09,4,4.133333,4.0,-0.133333,0.0
1,Hot Tea,14,2024-11-09,4,4.571429,4.0,-0.571429,0.0
2,Tumble,14,2024-11-10,3,4.642857,4.5,-1.642857,-1.5
3,All I Need,13,2024-11-08,5,4.846154,5.0,0.153846,0.0
4,Drive,13,2024-11-09,4,5.230769,5.0,-1.230769,-1.0
5,Arrow,13,2024-11-09,4,4.923077,4.0,-0.923077,0.0
6,Rockdale,13,2024-11-10,3,5.230769,5.0,-2.230769,-2.0
7,Red Bird,13,2024-11-10,3,4.923077,5.0,-1.923077,-2.0
8,Everything Must Go,13,2024-11-09,4,5.230769,5.0,-1.230769,-1.0
9,Creatures,12,2024-11-07,6,5.166667,5.0,0.833333,1.0


In [13]:
#Festival Goose
setlistdata_json = requests.get(setlist_url).json()
df = pd.DataFrame(setlistdata_json['data'])
setlist = df[(df['artist'] == "Goose") &(df['showyear'] >= 2020)]
festivalsetlist = setlist[(setlist['shownotes'].str.contains('festival', case=False, regex=False) |
                          (setlist['venuename'].str.contains('festival', case=False, regex=False)))]
festivalsetlist = festivalsetlist.drop(['uniqueid','permalink','artist','song_id','artist_id', \
                       'permalink','soundcheck','settype','transition_id','transition','isjamchart',\
                        'jamchart_notes','isverified','footnote','venue_id', 'showorder', 'opener',\
                        'tour_id','tourname', 'slug','original_artist','city','state',\
                        'country', 'showyear'], axis=1).reset_index(drop=True)
festivalsetlist.rename({'songname' : 'song'}, axis=1, inplace=True)

In [14]:
festival_stats = festivalsetlist.groupby('song').agg(
    count=('song', 'count')
).sort_values('count', ascending=False)

In [15]:
festivalsong_stats = festival_stats.merge(RicksNotebook, on='song')
festivalsong_stats = festivalsong_stats.loc[festivalsong_stats['current_gap']>2].reset_index(drop=True)
festivalsong_stats[['song','count','ltp_date','current_gap']].head(25)

NameError: name 'RicksNotebook' is not defined